In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [11]:
# Se placer dans le bon répertoire
import os
import sys

ROOT_DIR = os.path.abspath("../../")

/Users/mathisaulagnier/SignLanguageLetterRecognitionAI/src/notebooks
/Users/mathisaulagnier/SignLanguageLetterRecognitionAI/src/notebooks
/Users/mathisaulagnier/SignLanguageLetterRecognitionAI


In [3]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)
        self.X = torch.tensor(self.data.iloc[:, 1:].values, dtype=torch.float32)  # Les caractéristiques commencent à la colonne 1
        self.y = torch.tensor(self.data.iloc[:, 0].values, dtype=torch.long)     # La première colonne est la cible
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [13]:
# Définition des paramètres
input_size = 784 # nombre de pixels dans une image
hidden_size = 128 # nombre de neurones dans la couche cachée
num_classes = 26  # Nombre de lettres de l'alphabet (Sorties possibles)

learning_rate = 0.001 # Taux d'apprentissage
num_epochs = 10 # Nombre d'itérations sur le jeu de données
batch_size = 28 # Nombre d'images à traiter en même temps


# Se placer dans le bon répertoire
import os
import sys

ROOT_DIR = os.path.abspath("../../")

train_dataset_path = os.path.join(ROOT_DIR, "data/sign_mnist_train/sign_mnist_train.csv")
test_dataset_path = os.path.join(ROOT_DIR, "data/sign_mnist_test/sign_mnist_test.csv")

train_dataset = CustomDataset(train_dataset_path)
test_dataset = CustomDataset(test_dataset_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
